# AutoGen + LangChian + RAG + Function Call = Super AI Chabot

https://quickaitutorial.com/autogen-langchian-rag-function-call-super-ai-chabot/

`pip install langchain langchain-community langchain-chroma langchain-openai "pyautogen[retrievechat]" PyPDF2 chromadb`

## Imports

In [6]:
import os
from autogen import ConversableAgent
import autogen
from langchain_openai import AzureOpenAIEmbeddings, AzureOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Step 1: Configuration

In [2]:
autogen_llm_config = {"model": "gpt-4o", 
                      "api_type": "azure", 
                      "base_url": os.environ.get("AZURE_OPENAI_ENDPOINT"),
                      "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
                      "api_version": "2024-04-01-preview",
                      "temperature":0.7
                    }
autogen_llm_configlist = [autogen_llm_config]

llm_config_proxy = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list": autogen_llm_configlist,
    "request_timeout": 600
}

embedding_model = "text-embedding-3-large"
llm_model = "gpt-4o"

## Step 2: Read PDF

In [7]:
reader = PdfReader('../Resources/homematic-ip-anwenderhandbuch.pdf')
corpus = ''.join([p.extract_text() for p in reader.pages if p.extract_text()])

splitter =  RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,)
chunks = splitter.split_text(corpus)

embeddings = AzureOpenAIEmbeddings(chunk_size=1, model=embedding_model,)
vectors = Chroma.from_texts(chunks, embeddings)


## Step 3: Conversation Retrieval

In [9]:
qa = ConversationalRetrievalChain.from_llm(
    AzureOpenAI(temperature=0, model=llm_model),
    vectors.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", 
    return_messages=True),
)

## Step 4: Specify Config for the Assistant Agent.

In [10]:
llm_config_assistant = {
    "Seed" : 42,
    "temperature": 0,
        "functions": [
        {
            "name": "answer_PDF_question",
            "description": "Answer any PDF related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to PDF",
                    }
                },
                "required": ["question"],
            },
            
        }
    ],
    "config_list": autogen_llm_configlist,
    "timeout": 120,
}

## Step 5: Configuring the Assistant Agent

In [11]:
assistant = autogen.AssistantAgent(
            name="assistant",
            llm_config=llm_config_assistant,
            system_message="""You are a helpful assistant, Answer the question based on the context. 
                              Keep the answer accurate. Respond "Unsure about answer" if not sure about the answer. Please answer in german language"""
            
        )

## Step 6: Configuring the UserProxy Agent

In [22]:
user_proxy = autogen.UserProxyAgent(  
            name="user_proxy",
            human_input_mode="NEVER", 
            max_consecutive_auto_reply=10,
            code_execution_config={"work_dir": "coding", "use_docker": False},          
            #llm_config_assistant = llm_config_assistant,
            function_map={
                "answer_PDF_question": llm_config_assistant["functions"][0]["name"] # answer_PDF_question
            }
        )

## Step 7: Finally Chat

In [23]:
user_proxy.initiate_chat(
    assistant,
    message="""
Erstelle mir eine Zusammenfassung der Dokumentation mit maximal 100 Wörtern.
"""
)

user_proxy (to assistant):


Erstelle mir eine Zusammenfassung der Dokumentation mit maximal 100 Wörtern.


--------------------------------------------------------------------------------


TypeError: Completions.create() got an unexpected keyword argument 'Seed'